In [1]:
# run initial imports
from __future__ import print_function

import numpy as np
import pandas as pd

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

from scipy.integrate import odeint
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets

from datetime import datetime

In [2]:
NOW = datetime.now()
pDay=3 # number of corecast days - should not exceed 5 unless you are very careful in tuning, and understand how to interpert the results
input_dir = './data/'
covid_19_national_observations = pd.read_csv(input_dir + 'global/covid_19_national_observations.csv')

In [3]:
countries = covid_19_national_observations['Country/Region'].unique()
#print('Available Countries:', countries)

In [12]:

def analyze_country(country):
    c = country.lower().replace(' ', '_')
    #setup initial data frames
    input_dir = './data/countries/' + c + '/'
    cases = pd.read_csv(input_dir + 'covid_19_'+ c +'_cases.csv')
    totals = pd.read_csv(input_dir + './covid_19_' + c + '_totals.csv')
    
    #correct date parsing on some of the JH data
    cases['Date'] = pd.to_datetime(cases['Date'])
    totals['Date'] = pd.to_datetime(totals['Date'])
    
    cases = cases.sort_values('Date')
    totals = totals.sort_values('Date')
    
    cases = cases[cases['Confirmed'] > 0]
    
    likelyTotal = int(totals['Likely Cases C86'].max())
    confirmedActive = int(totals['Active Cases'].iloc[-1])
    recovered = int(totals['Recovered'].iloc[-1])
    dead = int(totals['Death'].iloc[-1])
    #print(likelyTotal, confirmedActive)
    
    # totals overall
    totals_state = totals[['Date', 'Active Cases', 'Recovered', 'Death']].melt(id_vars=['Date'], 
            value_vars=['Active Cases', 'Recovered', 'Death'], value_name="Population", var_name='Status')
    totals_chart = px.bar(totals_state, 
        x="Date", y="Population", color="Status", title=country+" Active, Recovered and Deaths by Date")
    totals_chart.show()

    # active cases only
    active_chart = px.bar(totals[totals['Active Cases'] > 0], 
        x="Date", y="Active Cases", color="Death Rate", 
        title=country+" Active Cases (confirmed - recovered - dead) - Current as of: "+str(NOW), 
    )
    active_chart.show()
    
    # confirmed chart
    confirmed_chart = px.bar(cases[cases['Confirmed'] > 0], 
        x="Date", y="Confirmed", color="Province/State", title=country+" Confirmed Cases")
    confirmed_chart.show()
    
    # now let's run the forecast with fbprophet
    fb_df = totals[['Date', 'Active Cases']].copy()
    fb_df = fb_df.sort_values('Date').reset_index(drop=True)
    fb_df.columns = ['ds','y']
    #print(fb_df)

    m = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, growth='linear')
    m.fit(fb_df)
    future = m.make_future_dataframe(periods=pDay)
    forecast = m.predict(future)
    py.init_notebook_mode()

    fig = plot_plotly(m, forecast, xlabel='Date', ylabel='Active Cases', uncertainty=True, figsize=(1100,600))  # This returns a plotly Figure
    fig.update_layout(title='Active '+country+' COVID-19 Cases and Forecast ('+str(pDay)+' day)')
    py.iplot(fig)

In [13]:
def f(country):
    analyze_country(country)
    
interactive_plot = interactive(f, country=countries)
output = interactive_plot.children[-1]
output.layout.height = '800px'
output.layout.width = '1200px'
interactive_plot

interactive(children=(Dropdown(description='country', options=('China', 'Japan', 'Korea, South', 'Taiwan*', 'T…